<a href="https://colab.research.google.com/github/dgromann/cl_intro_ws2024/blob/main/exercises/HomeExercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
import pandas as pd

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first 50 tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['train'].shuffle(seed=24).select(range(160, 200)).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    #unpack predictions
    logits, labels = eval_pred
    #convert logits to a predicted class indices; take index of max logit for each one
    predictions = np.argmax(logits, axis=-1)
    #calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model, #model to be trained
    args=arguments,#training arguments
    train_dataset=small_tokenized_dataset['train'],#training dataset
    eval_dataset=small_tokenized_dataset['val'], #validation dataset
    processing_class=tokenizer,#preprocess datset with tokenization
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.701200,0.698196,0.468750
2,0.680800,0.693926,0.468750
3,0.666000,0.686764,0.468750
4,0.659200,0.678429,0.656250
5,0.645800,0.677457,0.656250


TrainOutput(global_step=40, training_loss=0.6705888628959655, metrics={'train_runtime': 498.2497, 'train_samples_per_second': 1.284, 'train_steps_per_second': 0.08, 'total_flos': 18636507148416.0, 'train_loss': 0.6705888628959655, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6774572134017944,
 'eval_accuracy': 0.65625,
 'eval_runtime': 3.7579,
 'eval_samples_per_second': 8.515,
 'eval_steps_per_second': 0.532,
 'epoch': 5.0}

\### **hyperparameter optimization with OPTUNA**

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
import optuna
def objective(trial):
  learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
  weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True)
  num_train_epochs = trial.suggest_categorical(1, 5)
  batch_size = trial.suggest_int("batch_size", [8, 16, 32])

  trial_training_args = TrainingArguments(
    output_dir="sample_cl_trainer",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=8,
    num_train_epochs=num_train_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
  )

  trial_trainer = Trainer(
      model=AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2),
      args=trial_training_args,
      train_dataset=small_tokenized_dataset['train'],
      eval_dataset=small_tokenized_dataset['val'],
      processing_class=tokenizer,
      compute_metrics=compute_metrics
  )

  trial_trainer.train()
  eval_results = trial_trainer.evaluate()

  return eval_results["eval_accuracy"]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_params)

[I 2024-12-03 15:35:43,630] A new study created in memory with name: no-name-402bca6b-9580-4563-8a1c-2cacf504990f
[W 2024-12-03 15:35:43,634] Trial 0 failed with parameters: {'learning_rate': 4.5169344068097554e-06, 'weight_decay': 9.791995166729206e-05} because of the following error: TypeError("object of type 'int' has no len()").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-17-e02f0de47183>", line 5, in objective
    num_train_epochs = trial.suggest_categorical(1, 5)
  File "/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py", line 402, in suggest_categorical
    return self._suggest(name, CategoricalDistribution(choices=choices))
  File "/usr/local/lib/python3.10/dist-packages/optuna/distributions.py", line 515, in __init__
    if len(choices) == 0:
TypeError: object of type 'int' has no len()
[W 2024-12-03 15:35:43,637] Tria

TypeError: object of type 'int' has no len()

👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [ ]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-40")

def evaluation_on_model(eval_dataset):
  fine_tuned_model.eval() #set model to evaluation mode
  all_predictions = []
  all_labels = []

  #loop through each item and tokenize the text
  for item in eval_dataset:
    model_inputs = tokenizer(item['text'], padding=True, truncation=True, return_tensors="pt")
    labels = item['label']

    with torch.no_grad():
      logits = fine_tuned_model(**model_inputs).logits
      predictions = torch.argmax(logits)
      all_predictions.append(predictions)
      all_labels.append(labels)

  #convert predictions and labels to tensors
  all_predictions = torch.stack(all_predictions).cpu()
  all_labels = torch.tensor(all_labels).cpu()

  result = accuracy.compute(predictions=all_predictions, references=all_labels)
  return result

def prediction(dataset, model, tokenizer):
  results = []

  for item in dataset:
    text = item['text']
    model_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
      logits = model(**model_inputs).logits
      prediction = torch.argmax(logits)
      results.append({text, ["NEGATIVE", "POSITIVE"][prediction]})

  return pd.DataFrame(results)

fine_tuned_model.eval()
predicted_results = prediction(small_tokenized_dataset['val'], fine_tuned_model, tokenizer)
print(predicted_results)

                                                    0  \
0                                            NEGATIVE   
1   I have been a fan of Pushing Daisies since the...   
2   ... but the trouble of this production is that...   
3   This film moved me beyond comprehension, it is...   
4   Good action show, but nothing new. This one to...   
5                                            NEGATIVE   
6   The Revolt of the Zombies is not the worst mov...   
7   I have walked out of very few movies before th...   
8                                            POSITIVE   
9                                            NEGATIVE   
10                                           POSITIVE   
11                                           NEGATIVE   
12  I've no idea what dimwit from San Francisco ca...   
13  I love this movie. My only disappointment was ...   
14                                           POSITIVE   
15  I've read countless of posts about this game b...   
16                             

Run on Test set

In [ ]:
test_results = trainer.evaluate(eval_dataset=small_tokenized_dataset['test'])
print("Test set evaluation results:", test_results)

predictions = trainer.predict(test_dataset=small_tokenized_dataset['test'])

predicted_labels = np.argmax(predictions.predictions, axis=1)
test_texts = small_tokenized_dataset['test']['text']

label_map = {0: "NEGATIVE", 1: "POSITIVE"}
results_df = pd.DataFrame({
    "Text": test_texts,
    "Predicted Sentiment": [label_map[label] for label in predicted_labels]
})

print(results_df)

Test set evaluation results: {'eval_loss': 0.6866800785064697, 'eval_accuracy': 0.525, 'eval_runtime': 10.0157, 'eval_samples_per_second': 3.994, 'eval_steps_per_second': 0.3, 'epoch': 5.0}
                                                 Text Predicted Sentiment
0   If you made a genre flick in the late 80s, you...            POSITIVE
1   I really enjoyed this movie as a young kid. At...            NEGATIVE
2   As a rule, a Full Moon production logo is a wa...            POSITIVE
3   Terry Gilliam's fantastic, twisted story of a ...            POSITIVE
4   This is one of my all time favorites.<br /><br...            POSITIVE
5   Billed as a kind of sequel to The Full Monty, ...            POSITIVE
6   This was playing at our theater in Amsterdam a...            POSITIVE
7   The story and the show were good, but it was r...            NEGATIVE
8   Very interesting and moving documentary about ...            POSITIVE
9   Cuban Blood is one of those sleeper films that...            POSIT